<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/Evaluation-humanEval_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from types import FunctionType
import re
from tqdm import tqdm

In [ ]:
humanEval_dataset = load_dataset("openai_humaneval")
humanEval_dataset

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

In [ ]:
def execute_code(code, entry_point):
    exec(code, globals())

    try:
        check_func = globals()['check']

        candidate_func = globals().get(entry_point)
        if not isinstance(candidate_func, FunctionType):
            raise ValueError(f"{entry_point} is not a function")

        # check 함수로 검증 실행
        check_func(candidate_func)
        return True
    except AssertionError:
        print("Test failed: The generated solution did not pass the tests.")
        return False
    except KeyError as e:
        print(f"Error: Missing required function or variable - {e}")
        return False
    except ValueError as ve:
        print(f"Error: {ve}")
        return False
    except NameError as ne:
        print(f"Error: NameError - {ne}")
        return False
    except Exception as e:
        return str(e)

In [ ]:
passed = 0

for row in tqdm(humanEval_dataset['test'], desc="HumanEval dataset", unit="sample"):
    task_id = row['task_id']
    prompt = row['prompt']
    canonical_solution = row['canonical_solution']
    test_code = row['test']
    entry_point = row['entry_point']

    combined_code = prompt + canonical_solution
    combined_code += '\n' + test_code

    result = execute_code(combined_code, entry_point)

    if result:
        passed += 1
    else:
        print(f"Failed: {task_id}")

print(f"Passed: {passed}/{len(humanEval_dataset['test'])}")

HumanEval dataset: 100%|██████████| 164/164 [00:00<00:00, 383.54sample/s]

Passed: 164/164
